<a href="https://colab.research.google.com/github/Zywia/AD-projekt-2-/blob/master/projekt_2_AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bert-for-tf2

     |████████████████████████████████| 40kB 4.0MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=880d1ddd11f60e1182fec390e3e8579b6e1ea33751670b410d9adf2c964294e2
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=a497f66863dd06a314b55baf6c46eb2a51112a33652755f7a04757df82aca706
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=5d8898873e8a657e8751b4791db5c93d8e1c96e426240637fb9956ab34428074
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [3]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import pickle

from bert import bert_tokenization


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/1",
                            trainable=True)

In [6]:
train_data, test_data = tfds.load(name="civil_comments", split=["train", "test"], data_dir="/content/drive/My Drive/AD/civil_comments", batch_size=-1, as_supervised=True)



In [7]:
train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

In [8]:
negative_comments_positions = np.argwhere(train_labels == 1)[:,0]
not_all_positive_comments_positions = np.argwhere(train_labels == 0)[:negative_comments_positions.shape[0], 0]

negative_comments_positions_test = np.argwhere(test_labels == 1)[:,0]
not_all_positive_comments_positions_test = np.argwhere(test_labels == 0)[:negative_comments_positions_test.shape[0], 0]


not_all_positive_comments_positions_test.shape

(222,)

In [9]:
a = np.hstack ( (not_all_positive_comments_positions, negative_comments_positions))
b = np.hstack ( (not_all_positive_comments_positions_test, negative_comments_positions_test))
a = np.random.permutation(a)
b = np.random.permutation(b)
train_examples = train_examples[a]
train_labels = train_labels[a]

test_examples = test_examples[b]
test_labels = test_labels[b]


In [10]:
test_examples.shape

(444,)

In [13]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

--2020-07-04 18:41:06--  https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16775 (16K) [text/plain]
Saving to: ‘tokenization.py’

tokenization.py     100%[===================>]  16.38K  --.-KB/s    in 0.004s  

2020-07-04 18:41:07 (4.18 MB/s) - ‘tokenization.py’ saved [16775/16775]



In [14]:
import tokenization


sp_model_file = bert_layer.resolved_object.sp_model_file.asset_path.numpy()
tokenizer = tokenization.FullSentencePieceTokenizer(sp_model_file)


ModuleNotFoundError: ignored

In [40]:
bert_layer1 = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2",
                            trainable=True)

vocab_file = bert_layer1.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer1.resolved_object.do_lower_case.numpy()


tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

def tokenize_text(txt):
    tokenized_txt = tokenizer.tokenize(txt)
    tokenized_txt = ["[CLS]"] + tokenized_txt + ["[SEP]"]
    tokenized_txt = tokenizer.convert_tokens_to_ids(tokenized_txt)
    return tokenized_txt


In [41]:

train_examples = list(map(tokenize_text, train_examples))
test_examples = list(map(tokenize_text, test_examples))

In [42]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(train_examples, maxlen=100, padding='post')

max_seq_length = x_train.shape[1]
x_test = tf.keras.preprocessing.sequence.pad_sequences(test_examples, maxlen=100, padding='post', truncating='post')

train_mask = (x_train != 0).astype(int)
test_mask = (x_test != 0).astype(int)

train_segments = np.zeros(x_train.shape)
test_segments = np.zeros(x_test.shape)

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")

bert_inputs = [input_word_ids, input_mask, segment_ids]
pooled_output, sequence_output = bert_layer(bert_inputs)

final_layer = tf.keras.layers.Dense(1)
final_output = final_layer(pooled_output)

In [43]:
checkpoint_filepath = '/content/drive/My Drive/AD/checkpoint2'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False)


In [44]:
model = tf.keras.models.Model(inputs=bert_inputs, outputs=final_output)

In [45]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              # optimizer=tf.optimizers.Adam(learning_rate=1e-4), 
              optimizer=tf.optimizers.Adam(), 

              metrics=["accuracy"])



In [47]:
model.fit([x_train, train_mask, train_segments], train_labels, epochs=20, batch_size=32,
          validation_data=([x_test, test_mask, test_segments], test_labels))

Epoch 1/20
276/276 [==============================] - 178s 646ms/step - loss: 0.7063 - accuracy: 0.4982 - val_loss: 0.6989 - val_accuracy: 0.5000
Epoch 2/20
261/276 [===========================>..] - ETA: 9s - loss: 0.7055 - accuracy: 0.4982 

KeyboardInterrupt: ignored

In [48]:
import pandas as pd

text = pd.read_csv("notI.csv", names=["sentance"], index_col=False)
text = text.to_numpy()

print (text[:,0].tolist())
text = text[:,0].tolist()
text = list(map(tokenize_text, text))
text = tf.keras.preprocessing.sequence.pad_sequences(text, maxlen=100, padding='post')
mask = (text != 0).astype(int)
segments = np.zeros(text.shape)


['temporrytext that is absoulutlely not important', 'anotehre aboulutally not important text', 'you are ugly so ugly', 'hate speech']


In [49]:
model([text, mask, segments])

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.09061621],
       [0.09061621],
       [0.09061621],
       [0.09061621]], dtype=float32)>

In [ ]:
m = tf.keras.models.load_model('/content/drive/My Drive/AD/checkpoint'
)


In [ ]:


# print (text)
resoult = m([text, mask, segments])
print(resoult)

tf.Tensor(
[[0.18121138]
 [0.18121138]
 [0.18121138]
 [0.18121138]], shape=(4, 1), dtype=float32)
